# Тестовое задание по SQL. Case Place


1. Выведите самый продаваемый принт с количеством заказов за весь известный период.

Необходимо вывести:

-Артикул принта;

-Название принта №1;

-Количество заказов.


In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import random

# Создаем соединение с базой данных
conn = sqlite3.connect('print_store.db')
cursor = conn.cursor()

# Создаем таблицы
print("Создаем таблицы...")

# Таблица print_directory
cursor.execute('''
CREATE TABLE IF NOT EXISTS print_directory (
    print TEXT PRIMARY KEY,
    name_print_1 TEXT,
    name_print_2 TEXT
)
''')

# Таблица product_directory
cursor.execute('''
CREATE TABLE IF NOT EXISTS product_directory (
    name_store TEXT,
    nomenclature TEXT PRIMARY KEY,
    print TEXT,
    barcode TEXT,
    FOREIGN KEY (print) REFERENCES print_directory(print)
)
''')

# Таблица stocks_directory
cursor.execute('''
CREATE TABLE IF NOT EXISTS stocks_directory (
    date TEXT,
    nomenclature TEXT,
    warehouse TEXT,
    value_stocks INTEGER,
    FOREIGN KEY (nomenclature) REFERENCES product_directory(nomenclature)
)
''')

# Таблица orders_directory
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders_directory (
    date TEXT,
    nomenclature TEXT,
    orders_type TEXT,
    price REAL,
    quantity_product INTEGER,
    FOREIGN KEY (nomenclature) REFERENCES product_directory(nomenclature)
)
''')

# Заполняем таблицу print_directory
print("Заполняем print_directory...")
print_data = [
    ('art_1', 'Цветочек', 'Цветочек в поле'),
    ('art_2', 'Белое облачко', None),
    ('art_3', 'Машина', 'Спорткар'),
    ('art_4', 'Тигр', 'Тигр в цветочках')
]
cursor.executemany('INSERT INTO print_directory VALUES (?, ?, ?)', print_data)

# Заполняем таблицу product_directory
print("Заполняем product_directory...")
product_data = [
    ('Магазин 1', 'mag_11', 'art_1', 'Code_1'),
    ('Магазин 2', 'mag_21', 'art_1', 'Code_2'),
    ('Магазин 1', 'mag_12', 'art_2', 'Code_1'),
    ('Магазин 3', 'mag_31', 'art_2', 'Code_2'),
    ('Магазин 2', 'mag_22', 'art_1', 'Code_1'),
    ('Магазин 3', 'mag_32', 'art_3', 'Code_3'),
    ('Магазин 1', 'mag_13', 'art_4', 'Code_4'),
    ('Магазин 2', 'mag_23', 'art_3', 'Code_5'),
    ('Магазин 3', 'mag_33', 'art_4', 'Code_6')
]
cursor.executemany('INSERT INTO product_directory VALUES (?, ?, ?, ?)', product_data)

# Генерируем данные для stocks_directory
print("Заполняем stocks_directory...")
stocks_data = []
warehouses = ['Склад 1', 'Склад 2', 'Склад 3']
nomenclatures = ['mag_11', 'mag_12', 'mag_13', 'mag_21', 'mag_22', 'mag_23', 'mag_31', 'mag_32', 'mag_33']

for i in range(30):  # 30 дней данных
    date = (datetime(2024, 10, 1) + timedelta(days=i)).strftime('%Y-%m-%d')
    for nom in nomenclatures:
        warehouse = random.choice(warehouses)
        value = random.randint(1, 50)
        stocks_data.append((date, nom, warehouse, value))

cursor.executemany('INSERT INTO stocks_directory VALUES (?, ?, ?, ?)', stocks_data)

# Генерируем данные для orders_directory
print("Заполняем orders_directory...")
orders_data = []
order_types = ['Тип 1', 'Тип 2', 'Тип 3']
prices = [100, 120, 150, 175, 200, 250, 300]

# Добавляем предоставленные данные
provided_orders = [
    ('2024-10-17', 'mag_11', 'Тип 1', 150, 2),
    ('2024-10-17', 'mag_22', 'Тип 1', 120, 7),
    ('2024-10-16', 'mag_31', 'Тип 2', 300, 1),
    ('2024-10-15', 'mag_21', 'Тип 1', 175, 2),
    ('2024-10-15', 'mag_11', 'Тип 2', 150, 1)
]

for order in provided_orders:
    orders_data.append(order)

# Генерируем дополнительные данные
for i in range(50):  # 50 дополнительных заказов
    date = (datetime(2024, 10, 1) + timedelta(days=random.randint(0, 29))).strftime('%Y-%m-%d')
    nom = random.choice(nomenclatures)
    order_type = random.choice(order_types)
    price = random.choice(prices)
    quantity = random.randint(1, 10)
    orders_data.append((date, nom, order_type, price, quantity))

cursor.executemany('INSERT INTO orders_directory VALUES (?, ?, ?, ?, ?)', orders_data)

# Сохраняем изменения и закрываем соединение
conn.commit()

# Проверяем данные
print("\nПроверка данных:")
print("print_directory:")
for row in cursor.execute('SELECT * FROM print_directory LIMIT 5'):
    print(row)

print("\nproduct_directory:")
for row in cursor.execute('SELECT * FROM product_directory LIMIT 5'):
    print(row)

print("\nstocks_directory (первые 5 записей):")
for row in cursor.execute('SELECT * FROM stocks_directory LIMIT 5'):
    print(row)

print("\norders_directory (первые 5 записей):")
for row in cursor.execute('SELECT * FROM orders_directory LIMIT 5'):
    print(row)


print("\nБаза данных успешно создана! Файл: print_store.db")

Создаем таблицы...
Заполняем print_directory...
Заполняем product_directory...
Заполняем stocks_directory...
Заполняем orders_directory...

Проверка данных:
print_directory:
('art_1', 'Цветочек', 'Цветочек в поле')
('art_2', 'Белое облачко', None)
('art_3', 'Машина', 'Спорткар')
('art_4', 'Тигр', 'Тигр в цветочках')

product_directory:
('Магазин 1', 'mag_11', 'art_1', 'Code_1')
('Магазин 2', 'mag_21', 'art_1', 'Code_2')
('Магазин 1', 'mag_12', 'art_2', 'Code_1')
('Магазин 3', 'mag_31', 'art_2', 'Code_2')
('Магазин 2', 'mag_22', 'art_1', 'Code_1')

stocks_directory (первые 5 записей):
('2024-10-01', 'mag_11', 'Склад 2', 21)
('2024-10-01', 'mag_12', 'Склад 1', 18)
('2024-10-01', 'mag_13', 'Склад 3', 46)
('2024-10-01', 'mag_21', 'Склад 2', 37)
('2024-10-01', 'mag_22', 'Склад 2', 49)

orders_directory (первые 5 записей):
('2024-10-17', 'mag_11', 'Тип 1', 150.0, 2)
('2024-10-17', 'mag_22', 'Тип 1', 120.0, 7)
('2024-10-16', 'mag_31', 'Тип 2', 300.0, 1)
('2024-10-15', 'mag_21', 'Тип 1', 175.0

In [20]:
query = """
    SELECT prd.print as "Артикул принта",
        pd.name_print_1 as "Название принта №1",
        SUM(od.quantity_product) as "Количество заказов"
    FROM orders_directory od INNER JOIN product_directory prd ON od.nomenclature = prd.nomenclature
        INNER JOIN print_directory pd ON prd.print = pd.print
    GROUP BY prd.print, pd.name_print_1
    ORDER BY "Количество заказов" DESC
    LIMIT 1
"""


print("Результат запроса:")
result = cursor.execute(query).fetchone()
if result:
    print(f"Артикул: {result[0]}, Название: {result[1]}, Количество заказов: {result[2]}")
else:
    print("Нет данных")


Результат запроса:
Артикул: art_1, Название: Цветочек, Количество заказов: 115


In [21]:
conn.close()

2. Есть три таблицы:
- customers (id, name, region)
- orders (id, customer_id, order_date, total_amount)
- order_items (id, order_id, product_id, quantity, price)

Составьте запрос, который вернёт для каждого клиента:
1. Общее количество заказов.
2. Сумму всех заказов.
3. Средний чек (сумма заказа / количество заказов).
4. Дату первого заказа.


In [27]:
import sqlite3
import random
from datetime import datetime, timedelta
from faker import Faker

fake = Faker()

conn = sqlite3.connect('ecommerce_advanced.db')
cursor = conn.cursor()

# Создаем расширенные таблицы
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    region TEXT NOT NULL,
    registration_date DATE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    category TEXT NOT NULL,
    base_price REAL NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total_amount REAL NOT NULL,
    status TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    unit_price REAL NOT NULL,
    total_price REAL NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
)
''')

# Создаем продукты
products_data = [
    ('MacBook Pro 13"', 'Electronics', 1299.99),
    ('iPhone 15', 'Electronics', 799.99),
    ('Samsung Galaxy Tab', 'Electronics', 299.99),
    ('Sony Headphones', 'Audio', 199.99),
    ('Logitech Keyboard', 'Accessories', 79.99),
    ('Wireless Mouse', 'Accessories', 39.99),
    ('4K Monitor', 'Electronics', 349.99),
    ('Office Chair', 'Furniture', 199.99),
    ('Webcam HD', 'Electronics', 89.99),
    ('External SSD 1TB', 'Storage', 129.99)
]

cursor.executemany('INSERT INTO products (name, category, base_price) VALUES (?, ?, ?)', products_data)

# Создаем клиентов
customers_data = []
regions = ['North America', 'Europe', 'Asia', 'South America', 'Africa', 'Australia']

for _ in range(100):
    name = fake.name()
    email = fake.email()
    region = random.choice(regions)
    reg_date = fake.date_between(start_date='-2y', end_date='today')
    customers_data.append((name, email, region, reg_date))

cursor.executemany('INSERT INTO customers (name, email, region, registration_date) VALUES (?, ?, ?, ?)', customers_data)

# Создаем заказы
customer_ids = [row[0] for row in cursor.execute('SELECT id FROM customers')]
product_ids = [row[0] for row in cursor.execute('SELECT id FROM products')]
statuses = ['completed', 'processing', 'shipped', 'delivered', 'cancelled']

orders_data = []
order_items_data = []

for order_id in range(1, 301):  # 300 заказов
    customer_id = random.choice(customer_ids)
    order_date = fake.date_between(start_date='-1y', end_date='today')
    status = random.choice(statuses)
    
    # Создаем товары в заказе
    total_amount = 0
    items_count = random.randint(1, 6)
    
    for _ in range(items_count):
        product_id = random.choice(product_ids)
        base_price = cursor.execute('SELECT base_price FROM products WHERE id = ?', (product_id,)).fetchone()[0]
        quantity = random.randint(1, 3)
        unit_price = round(base_price * random.uniform(0.8, 1.2), 2)  # ±20% от базовой цены
        item_total = unit_price * quantity
        total_amount += item_total
        
        order_items_data.append((order_id, product_id, quantity, unit_price, item_total))
    
    orders_data.append((customer_id, order_date, round(total_amount, 2), status))

cursor.executemany('INSERT INTO orders (customer_id, order_date, total_amount, status) VALUES (?, ?, ?, ?)', orders_data)
cursor.executemany('INSERT INTO order_items (order_id, product_id, quantity, unit_price, total_price) VALUES (?, ?, ?, ?, ?)', order_items_data)

conn.commit()

# Статистика
print("Расширенная база данных создана!")
print(f"Клиентов: {len(customer_ids)}")
print(f"Заказов: {300}")
print(f"Товаров: {len(product_ids)}")
print(f"Позиций заказов: {len(order_items_data)}")



Расширенная база данных создана!
Клиентов: 200
Заказов: 300
Товаров: 20
Позиций заказов: 1057


/var/folders/lb/7x2fn5_57wj2tp98418jylg80000gp/T/ipykernel_67033/561320447.py:82: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany('INSERT INTO customers (name, email, region, registration_date) VALUES (?, ?, ?, ?)', customers_data)
/var/folders/lb/7x2fn5_57wj2tp98418jylg80000gp/T/ipykernel_67033/561320447.py:113: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany('INSERT INTO orders (customer_id, order_date, total_amount, status) VALUES (?, ?, ?, ?)', orders_data)


In [49]:
query = """
    SELECT c.id as customer_id,
        COUNT(o.id) as orders_cnt,
        COALESCE(SUM(o.total_amount), 0) as orders_sum,
        CASE WHEN COUNT(o.id) = 0 THEN 0
        ELSE ROUND(COALESCE(SUM(o.total_amount), 0) / COUNT(o.id), 2)
        END as mean_amount,
        MIN(order_date) as first_date
    FROM customers c LEFT JOIN orders o ON c.id = o.customer_id
    GROUP BY c.id
    ORDER BY c.id
"""


print("Результат запроса:")
# result = cursor.execute(query).fetchone()
result = cursor.execute(query).fetchall()
# if result:
#     print(f"Артикул: {result[0]}, Название: {result[1]}, Количество заказов: {result[2]}")
# else:
#     print("Нет данных")
print(pd.DataFrame(result).head())

Результат запроса:
   0  1         2        3           4
0  1  3   8988.36  2996.12  2025-03-31
1  2  7   9660.45  1380.06  2024-12-05
2  3  5  10947.22  2189.44  2024-10-28
3  4  2   5237.15  2618.57  2024-12-05
4  5  5  18355.68  3671.14  2024-10-14


In [ ]:
conn.close()